## Music similarity search using ConvNet on STFT spectrograms

Below example uses 4th model [(ConvNet on STFT spectrograms)](https://gitlab.crowdai.org/gg12/WWWMusicalGenreRecognitionChallenge) from [CrowdAI Learning to Recognize Musical Genre competition](crowdai.org/challenges/www-2018-challenge-learning-to-recognize-musical-genre) to extract features from music.

Music comes from [GTZAN music dataset](http://marsyasweb.appspot.com/download/data_sets/).

In [ ]:
# run this in case you use Google's Colab
!pip install git+http://github.com/lambdaofgod/findkit
!mkdir -p ../resources

In [ ]:
import numpy as np
import scipy

import librosa
import librosa.display
import glob
import keras

from findkit.feature_extractor import KerasFeatureExtractor
from findkit.index import AnnoyIndex

import matplotlib.pyplot as plt
%matplotlib inline


from IPython.display import Audio, display

In [ ]:
def stft3_quick(file_path):
    res = []

    Y, sr = librosa.load(file_path)

    for i in np.arange(0.0, 9.5, 0.5):
        y = Y[int(66150 * i):min(len(Y), int(66150 * (i + 1)))]
        if y.shape[0] < 66150:
            y = np.pad(y, (0, 66150 - y.shape[0]), 'constant')
        res.append(np.abs(librosa.stft(y, n_fft=1024, window=scipy.signal.hann, hop_length=512))[:, :128].T)
    res = np.array(res)
    return res, file_path


def stft3_quick_iter(paths):
    for p in paths:
        stft, path = stft3_quick(p)
        yield np.expand_dims(stft, -1)


def truncate_keras_model(model, layer_name):
    output = model.get_layer(layer_name).output
    return keras.models.Model(model.input, output)


def play_track(path):
    track, fs = librosa.load(path)
    librosa.output.write_wav('/tmp/example.wav', track, fs)
    print(path)
    display(Audio('/tmp/example.wav'))
    

def display_track(path):
    track, fs = librosa.load(path)
    librosa.output.write_wav('/tmp/example.wav', track, fs)
    print(path)
    display(Audio('/tmp/example.wav'))
    spec = librosa.stft(track, n_fft=1024)
    librosa.display.specshow(np.abs(spec));

## Setup data

In [ ]:
MODEL_PATH = '../resources/keras_genre_recognition_simple_model.h5'
DATASET_PATH = '../resources/genres.tar.gz'

In [ ]:
# Download GTZAN dataset
!wget -nc http://opihi.cs.uvic.ca/sound/genres.tar.gz -O $DATASET_PATH
!tar -xvf ../resources/genres.tar.gz -C ../resources
# Download Keras model
!wget -nc https://gitlab.crowdai.org/gg12/WWWMusicalGenreRecognitionChallenge/raw/master/best_rescnnqstft3_b128.h5 -O $MODEL_PATH

## Setup model

In [ ]:
full_model = keras.models.load_model(MODEL_PATH)
full_model.summary()

In [ ]:
paths = glob.glob('../resources/**/*.au', recursive=True)

## Play example track and display its STFT

In [ ]:
example_path = paths[0]
display_track(example_path)

### Extract features

In [ ]:
model = truncate_keras_model(full_model, 'dense_2')
feature_extractor = KerasFeatureExtractor(model)

In [ ]:
%store -r features

In [ ]:
features = feature_extractor.extract_features_generator(stft3_quick_iter(paths), steps=1000, verbose=1, use_multiprocessing=True)
%store features

### Setup index

In [ ]:
index = AnnoyIndex(features, 50, metric='angular')

In [ ]:
indices, distances = index.find_similar(features[0], 5)

In [ ]:
for i, dist in zip(indices, distances):
    print('distance: ', dist)
    play_track(paths[i // 19])